In [1]:
# https://github.com/swar/nba_api/issues/124

In [2]:
import pandas as pd
from time import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playerawards

In [3]:
from nba_api.stats.library import http

print(http.STATS_HEADERS)

{'Host': 'stats.nba.com', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0', 'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate, br', 'x-nba-stats-origin': 'stats', 'x-nba-stats-token': 'true', 'Connection': 'keep-alive', 'Referer': 'https://stats.nba.com/', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache'}


In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
# function which collects player seasons and drops before modern 3 point era and insufficient data

def collect_seasons(id):
    season_filter = 1998 # the first year of the modern 3 point era

    career = playercareerstats.PlayerCareerStats(player_id=id)
    df = career.get_data_frames()[0]

    df['SEASON_ID'] = df['SEASON_ID'].apply(lambda x: int(x[:4]) + 1) #make seasons numeric type
    df = df[df['SEASON_ID'] >= season_filter] # drop any seasons before modern era

    return df

In [6]:
# function which gets player awards

# fun fact and test case: dominique wilkins in 94 made an All-NBA team
# but played for two teams that year

def collect_awards(id):
    awards = playerawards.PlayerAwards(player_id=id)

    df = awards.get_data_frames()[0]

    df = df[df['DESCRIPTION'] == 'All-NBA']
    df = df[['SEASON', 'ALL_NBA_TEAM_NUMBER']]

    df['SEASON'] = df['SEASON'].apply(lambda x: int(x[:4]) + 1) #make seasons numeric type

    return df

In [7]:
# function which joins the awards and seasons

def stat_join(seasons, awards):
    df = seasons.join(awards.set_index('SEASON'), on='SEASON_ID')
    df['ALL_NBA_TEAM_NUMBER'] = df['ALL_NBA_TEAM_NUMBER'].fillna(0)
    return df

In [10]:
# function which loops through each player

nba_players = players.get_players()

In [13]:
len(nba_players)

4900

In [12]:
nba_players[0]

{'id': 76001,
 'full_name': 'Alaa Abdelnaby',
 'first_name': 'Alaa',
 'last_name': 'Abdelnaby',
 'is_active': False}

In [17]:
len(nba_players[4892: 4900])

8

In [24]:
def collect_chunk(nba_players, start_point, chunk_size):
    final_df = pd.DataFrame()
    for count, player in enumerate(nba_players[start_point:start_point + chunk_size]):
        id = player['id']
        name = player['full_name']

        start = time()
        print(f"Processing {name}...")

        seasons = collect_seasons(id)
        awards = collect_awards(id)
        df = stat_join(seasons, awards)
        df['NAME'] = player['full_name']

        if count == 0:
            final_df = df
        else:
            final_df = pd.concat([final_df, df])
        
        end = time()
        print(f"Took {end - start} seconds to finish")

    final_df.to_csv(f'../data/chunks/players_{start_point}_to_{start_point + chunk_size}.csv')
    return

In [25]:
chunk_size = 100
start_point = 1000

while start_point < len(nba_players):
    if start_point + chunk_size > len(nba_players):
        chunk_size = len(nba_players) - start_point
    print(f'collecting chunk {start_point} through {start_point + chunk_size}')
    collect_chunk(nba_players, start_point, chunk_size)
    start_point = start_point + chunk_size

collecting chunk 1000 through 1100
Processing Mark Davis...
Took 2.8120172023773193 seconds to finish
Processing Mark Davis...
Took 1.2777395248413086 seconds to finish
Processing Mel Davis...


/tmp/ipykernel_4467/3617577825.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 1.4683737754821777 seconds to finish
Processing Michael Davis...
Took 2.2348341941833496 seconds to finish
Processing Mike Davis...


/tmp/ipykernel_4467/3617577825.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 1.3064830303192139 seconds to finish
Processing Monti Davis...
Took 5.316437482833862 seconds to finish
Processing Paul Davis...
Took 2.503627061843872 seconds to finish
Processing Ralph Davis...


...

/tmp/ipykernel_4467/3617577825.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 6.854215860366821 seconds to finish
Processing Bob Doll...


/tmp/ipykernel_4467/3617577825.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 6.010287761688232 seconds to finish
Processing James Donaldson...
Took 1.62898588180542 seconds to finish
Processing Luka Doncic...


KeyboardInterrupt: 